<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T055605Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbed536196ddbefc905539fcc2633adf752808daebae3699d48e1b14ce81b341b2c99b5aed28ab13187be50fb3bef07b6d811ad2321b1b0e28e1d11c3960199db000778d01c35644b5f55eeac4e4f62b8d1d7d784e7bb61dba49337965a60183409c5391d480c197faff0d897494bfb3fbf09be02d7d70412f45a02b32a3cd8d44bbd12c7a8aeeea18bfa46f193caf0ff1f1faac2bf69bf196f75d929b6c775a36850732c2a33413cf5a6b1615c68fb01fc2eedcdd07c1cf668c33638d9c46425315bd34ab212b1d757ed45590dc9495f535b84f383d737930870d9510ee0d87b074afffe9e3e1a6ef6abe5dacc12aa29738da3301bade988ed92a27e0a9b817c,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T055605Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5207095bf44aec03b51c2537e12b23e82acd42a1a59e5dab728d33273324df2c81d1c720c5911129b54b02736e49bd27bdae3740f84495d666e1a20b3cc13c7a7135b035c31b6424ac1bfe99dd7e139ad38563fcada9091fc0cf2698d6ca6355110970910c767996fed8c0f2ae47e0be6f08ae0bb922ab12277fba24bd4ea607d0e7c49931f9fae14da925ec3faab4773fda9132d4c09291ca81cb161976da37e46b8a6b82cc4c8135dfc98c83e44942428138044014d976b7ef6790784c47efba80be8f9149620dd6bafa7064ee8e48a5eefd916f2e8553d29c52b592b41ecdae5bdca1f5b88a163562f243b585ffe3e17f9b76800d5c3e4d2e49b0ffc2ddf5,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T055605Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dce56110ac732e93976aec6db4b90a6fcc6a8deb1921aaed6692ab4a6b9d9d86d9da41639e0b137e352750f61dd19b7579ff458764f89da70f60680cb00c52eeb9d64e755eebc477724cee5a3bb730cdf32cc2bb0963b17a4f2c4ce342db6b6e209202cec8bbc22ab0b57f0f9971327982b426e2b00afb34b7fc6fcff3381e08d3674b170375aeb14bd8aed6e14a58ef228dc1598b954e87fff6499ec41a719ea13449cb0e07777ad467715356a30c4fc85c4aeee0bdfbc59a3cb1932975fd926e0d5d24ddeeeddaabc3b826cd21a49fb4bb2d70a685d195bca92e549258a2829222f68ffa9b8511653c288a3deafe7770b7f2414f3e06a8c83e99ecb80d063a1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) sponsored by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models y les code changes

## Requirements

In [ ]:
!pip install langchain==0.0.209
!pip install chromadb==0.3.26
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.3
    Uninstalling pydantic-2.5.3:
      Successfully uninstalled pydantic-2.5.3
  Attempting uninstall: dataclasses-json
    Found existing installation: dataclasses-json 0.6.4
    Uninstalling dataclasses-json-0.6.4:
      Successfully uninstalled dataclasses-json-0.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires pydantic>=2, but 

In [ ]:
#base
import os
import json
import re
import csv
import pandas as pd
from typing import *
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

2024-04-02 04:30:01.239549: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 04:30:01.239678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 04:30:01.370325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Classes

### Questions and answers data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        qa_data = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        qa_data = qa_data[qa_data['answer_score'] > 5]
        df_qa_data = qa_data[qa_data['Tag']=='python']
        return df_qa_data

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        df_qa_data = self.qa_data()
        df_qa_data['Answer'] = df_qa_data['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        data = [
            f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data


### Python Enhancement Proposals data

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

In [ ]:
class Embeder:
    """
    Creates embeddings (numerical representations) of text documents for various tasks like semantic search and
    similarity comparison. Provides flexibility to choose between a large model for more comprehensive
    embeddings or a smaller, faster model.

    Attributes:

    large (bool): Flag indicating whether to use the large embedding model (default: True).
    model (str): Name of the embedding model to use.
    device (str): Device to use for computations (default: "cuda").
    """
    def __init__(
            self,
            large: bool = True
    ):
        """
        init(self, large: bool = True) -> None
            Initializes the class with the specified model size preference.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device: str = "cuda"
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(
                model_name=self.model
            )
        return embed

    def run(
            self,
            docs_list: list,
    ):
        """
        run(self, docs_list: list) -> List[List[float]]:
            Generates embeddings for the provided list of documents.
            Returns a list of lists, where each inner list represents the embedding vector for a document.
        """
        if docs_list is None:
            docs_list = ['']
        embed = self.instructor()

        return embed.embed_documents(docs_list)

In [ ]:
class Settings:
    user_secrets = UserSecretsClient()
    CHUNK_SIZE = user_secrets.get_secret("CHUNK_SIZE")
    CHUNK_OVERLAP = user_secrets.get_secret("CHUNK_OVERLAP")
    CHROMA_NAME_INDEX = user_secrets.get_secret("CHROMA_NAME_INDEX")
    K = user_secrets.get_secret("K")
    NN_THRESHOLD = user_secrets.get_secret("NN_THRESHOLD")

In [ ]:
embeder = Embeder()
settings = Settings()
class Retriever:
    """ retriever
    Retrieves the embeddings from a vectorial database
    """
    def __init__(self, data: list[dict]):
        """ initialize the retriever
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.data = data

    def load(self):
        """
        Loads documents from the initialized data and returns them as a list of Document objects.
        """
        documents = []
        for obj in self.data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def set(self):
        """
         update the vectorial database
        """
        data = self.load()
        splitter = self.text_splitter
        documents = splitter.split_documents(data)
        instructor = embeder.instructor()
        vector_db = Chroma.from_documents(
            documents=documents,
            embedding=instructor,
            persist_directory=self.index_name
        )
        vector_db.persist()

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
    ):
        """ retrieve the 2 more similar text chunks based on the message given
        """
        vectorial_db = Chroma(
            embedding_function=embeder.instructor(),
            persist_directory=self.index_name
        )
        res = vectorial_db.similarity_search_with_score(message, k=k)
        return [vector[0].page_content for vector in res if vector[1] < threshold]

## Data

### RAG Data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

corpus lenght: 644 
section <n> lenght: 85988 
sample text:

 ing and distributing wheels with this platform tag, and that pip
support downloading and installing these packages on compatible platforms.


Rationale
Currently, distribution of binary Python extensions for Windows and OS X is
straightforward. Developers and packagers build wheels (PEP 427, PEP 491),
which are
assigned platform tags such as win32 or macosx_10_6_intel, and upload
these wheels to PyPI. Users can download and install these wheels using tools
such as pip.
For Linux, the situation is much more delicate. In general, compiled Python
extension modules built on one Linux distribution will not work on other Linux
distributions, or even on different machines running the same Lin
CPU times: user 49.8 s, sys: 687 ms, total: 50.5 s
Wall time: 1min 29s


In [ ]:
%%time
peps = Peps()
data = peps.jsonl_format()
data[-1]['text'][:2000]

CPU times: user 51.4 s, sys: 740 ms, total: 52.2 s
Wall time: 1min 20s


'\n\nPython Enhancement Proposals\n\nPython » \nPEP Index » \nPEP 507\n\n\n\n\n\nToggle light / dark / auto colour theme\n\n\n\n\nPEP 507 – Migrate CPython to Git and GitLab\n\nAuthor:\nBarry Warsaw <barry at python.org>\nStatus:\nRejected\nType:\nProcess\nCreated:\n30-Sep-2015\nPost-History:\n\nResolution:\nCore-Workflow message\n\n\n\nTable of Contents\nAbstract\nRationale\nVersion Control System\nRepository Hosting\nCode Review\nGitLab merge requests\n\n\nCriticism\nX is not written in Python\nMercurial is better than Git\nCPython Workflow is too Complicated\n\n\nOpen issues\nReferences\nCopyright\n\n\n\nAbstract\nThis PEP proposes migrating the repository hosting of CPython and the\nsupporting repositories to Git.  Further, it proposes adopting a\nhosted GitLab instance as the primary way of handling merge requests,\ncode reviews, and code hosting.  It is similar in intent to PEP 481\nbut proposes an open source alternative to GitHub and omits the\nproposal to run Phabricator.  As 

### fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_formatted_qa()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][1306:2000]
)

Python Questions loaded: 33313 
 
Sample question-answer:
 
CPU times: user 18.5 s, sys: 1.11 s, total: 19.6 s
Wall time: 32.1 s


## RAG

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["hello I'm goku"])[0]
print(len(vec))
vec[:10]

load INSTRUCTOR_Transformer
max_seq_length  512
768


[-0.011055625043809414,
 -0.01609027571976185,
 -0.010001139715313911,
 -0.01644662767648697,
 0.030971292406320572,
 0.024415701627731323,
 0.011769121512770653,
 -0.0039945864118635654,
 0.0033679513726383448,
 0.01626904122531414]

In [ ]:
%%time
retriever = Retriever(data)
retriever.set()

load INSTRUCTOR_Transformer
max_seq_length  512


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
retriever = Retriever(data)
question = "what is a List comprehensions and what is it's relation with map() and filter()?"
retriever.query(question)

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0